# PyTorch BYOS

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [ ]:
!/bin/bash ./utils/setup.sh

## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [1]:
import os
import sagemaker
print(sagemaker.__version__)

from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mae'

role = sagemaker.get_execution_role()

2.77.1


In [2]:
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Fri Mar 18 04:47:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   41C    P0    39W / 300W |   6382MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Download pretrained model & Training data

You can also prepare your own image classification dataset, please organize it in the following format:

Dataset_customer/

    train/
        class1/
            0.jpg
            1.jpg
            ...
        class2/
        ...
    val/
        class1/
            0.jpg
            1.jpg
            ...
        class2/
        ...
        

In [3]:
!cd pretrained_model && wget -nc https://dl.fbaipublicfiles.com/mae/pretrain/mae_pretrain_vit_base.pth
!aws s3 cp s3://datalab2022/chinesefoodnet/chinesefoodnet_small.zip ./datasets
!cd datasets && unzip chinesefoodnet_small.zip > /dev/null 2>&1

--2022-03-18 03:22:02--  https://dl.fbaipublicfiles.com/mae/pretrain/mae_pretrain_vit_base.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343249461 (327M) [binary/octet-stream]
Saving to: ‘mae_pretrain_vit_base.pth’

100%[======================================>] 343,249,461 40.2MB/s   in 7.9s   

2022-03-18 03:22:11 (41.3 MB/s) - ‘mae_pretrain_vit_base.pth’ saved [343249461/343249461]

download: s3://datalab2022/chinesefoodnet/ChineseFoodNet.zip to datasets/ChineseFoodNet.zip


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [3]:
WORK_DIRECTORY1 ='./pretrained_model'
WORK_DIRECTORY2 ='./datasets'
print('s3://{}/{}'.format(bucket, prefix))
data_location = sagemaker_session.upload_data(WORK_DIRECTORY1, key_prefix=prefix)
print(data_location)
data_location = sagemaker_session.upload_data(WORK_DIRECTORY2, key_prefix=prefix)
data_location = 's3://{}/{}'.format(bucket, prefix)
print(data_location)
inputs = {'weights': data_location+'/pretrained_model/', 'data_path':data_location+'/datasets/'}
print(inputs)

{'weights': 'file:///home/ec2-user/SageMaker/mae/pretrained_model/', 'data_path': 'file:///home/ec2-user/SageMaker/mae/datasets/'}


## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


## SageMaker Training using GPU instance

In [12]:
git_config = {'repo': 'https://github.com/HaoranLv/mae.git', 'branch': 'main'}

hyperparameters = {'data_path': '/opt/ml/input/data/data_path/chinesefoodnet_small', 
                   'model': 'vit_base_patch16', 
                   'finetune': '/opt/ml/input/data/weights/mae_pretrain_vit_base.pth',
                   'output_dir': '/opt/ml/model/',
                   'accum_iter': 4, 'batch_size': 32, 'epochs': 1, 'weight_decay': 0.05,'nb_classes':2,
                   'drop_path': 0.1, 'mixup':0.8, 'cutmix':1.0, 'reprob':0.25, 'blr':5e-4, 'layer_decay':0.65}  # set batch and workers to 1, becasue of shared memory issue in local mode
#                    'name': 'tutorial', 'img': 640, 'batch': 8, 'epochs': 5, 'workers': 1, 'device': '0,1,2,3,4,5,6,7'}  # set batch and workers to 1, becasue of shared memory issue in local mode
instance_type = 'ml.p3.2xlarge'
estimator = PyTorch(entry_point='main_finetune.py',
                            source_dir='.',
                            git_config=git_config,
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.7.1',
                            py_version='py36',
                            script_mode=True,
                            instance_count=1,  # 1 or 2 or ...
                            instance_type=instance_type)

estimator.fit(inputs)

Cloning into '/tmp/tmp27ttpd_j'...
Already on 'main'


Your branch is up to date with 'origin/main'.
2022-03-18 05:39:31 Starting - Starting the training job...
2022-03-18 05:39:58 Starting - Preparing the instances for trainingProfilerReport-1647581971: InProgress
.........
2022-03-18 05:41:17 Downloading - Downloading input data...
2022-03-18 05:41:54 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-03-18 05:45:07,003 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-03-18 05:45:07,028 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-03-18 05:45:07,037 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-03-18 05:45:07,499 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Attempting uninstall: import

In [13]:
# training_job_name = estimator.latest_training_job.name
training_job_name = 'pytorch-training-2022-03-18-05-39-29-920'
print(training_job_name)

pytorch-training-2022-03-18-05-39-29-920


# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [14]:
!rm -rf model.tar.gz
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!tar -xvf model.tar.gz

download: s3://sagemaker-us-west-2-847380964353/pytorch-training-2022-03-18-05-39-29-920/output/model.tar.gz to ./model.tar.gz
checkpoint-0.pth
log.txt


In [43]:
!rm -rf model-pytorch.tar.gz
# !cp checkpoint-0.pth model/
!cd model && tar -czvf ../model-pytorch.tar.gz *

!aws s3 cp model-pytorch.tar.gz s3://$bucket/$training_job_name/output/model-pytorch.tar.gz

checkpoint-0.pth
code/
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/inference-checkpoint.py
code/.ipynb_checkpoints/models_vit-checkpoint.py
code/models_vit.py
code/inference.py
code/requirements.txt
code/__pycache__/
code/__pycache__/models_vit.cpython-37.pyc
upload: ./model-pytorch.tar.gz to s3://sagemaker-us-west-2-847380964353/pytorch-training-2022-03-18-05-39-29-920/output/model-pytorch.tar.gz


In [44]:
# instance_type = 'local'
instance_type = 'ml.g4dn.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-pytorch.tar.gz'.format(bucket, training_job_name), role=role,
                             entry_point='inference.py', framework_version='1.7.1', py_version='py36', model_server_workers=1)  # TODO set model_server_workers=1 to avoid torchhub bug
predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

-----------!

# Invoking the endpoint

In [46]:
import cv2
image = cv2.imread('demo/000000.jpg')

print('image:', type(image), image.shape, image.dtype)
outputs = predictor.predict(image)
print('outputs: ', outputs)

image: <class 'numpy.ndarray'> (480, 480, 3) uint8
outputs:  (768,)


## Deploy the trained model to SageMaker Endpoint Serverless (Preview)

In [ ]:
!aws s3 cp model-pytorch.tar.gz s3://$bucket/$training_job_name/output/model.tar.gz

In [ ]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=1,
)

image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sagemaker_session.boto_region_name,
    version="1.9.1",
    py_version="py38",
    instance_type="ml.m5.large",
    image_scope="inference",
)
print(image_uri)

estimator = PyTorch.attach(training_job_name=training_job_name)
serverless_predictor = estimator.deploy(serverless_inference_config=serverless_config, image_uri=image_uri, entry_point='inference.py', model_server_workers=1)

# pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-pytorch.tar.gz'.format(bucket, training_job_name), role=role,
#                              entry_point='inference.py', framework_version='1.9.1', py_version='py38', model_server_workers=1)  # TODO set model_server_workers=1 to avoid torchhub bug
# serverless_predictor = pytorch_model.deploy(serverless_inference_config=serverless_config, image_uri=image_uri)

In [ ]:
import cv2
image = cv2.imread('data/images/inference/bus.jpg')

# print('image:', type(image), image.shape, image.dtype)
outputs = serverless_predictor.predict(image)
print('outputs: ', outputs)

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
# estimator.delete_endpoint()
predictor.delete_endpoint()
serverless_predictor.delete_endpoint()